# TIES451 course. Programming assignment 1 #

Konstantin Sakharovskiy. Sources used:
 - Lecture slides
 - Recommended papers
 - Several open-source GA implementations

## Task1 binary-coded Genetic Algorithm for continuous function ##

**Step 1.** - make all required imports, defining functions and constraints 

In [1]:
# - random integer to generate initial population and select genes in crossover
# - random real to handle mutation and crossover probability

from numpy.random import randint
from numpy.random import rand

# objective function
def f(x):
    return x[0] + x[1] - 2 * x[0] ** 2 - x[1] ** 2 + (x[0] * x[1])

#  Rosenbrock function for testing the algorithm, (optima = 0 at [1,1])
def rosenbrock(x):
    return 100 * ((x[1] - x[0] ** 2)) ** 2 + (1 - x[0]) ** 2 

# define range for input
OBJ_BOUNDS = [[1.0, 5.0], [1.0, 5.0]]
ROSEN_BOUNDS = [[-10.0, 10.0], [-10.0, 10.0]]

# define the total iterations
OBJ_ITER = 50
ROSEN_ITER = 5000 # It has been determined by experience that this function requires more iterations

# bits per variable
OBJ_BITS = 8
ROSEN_BITS = 16

# define the population size
OBJ_POP = 10
ROSEN_POP = 20

# crossover rate
OBJ_CROSS = 0.8
ROSEN_CROSS = 0.8

# mutation rate
OBJ_MUT = 0.06
ROSEN_MUT = 0.06

**Step 2.** Given that the optimizing functions are in real numbers and the algorithm
encodes in binary, we need an auxiliary decoding function (not needed in differential evolution)

In [2]:
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded



**Step 3.** code all necessary steps in the Evolutionary Generational Cycle:

Selection -> Crossover -> Mutation -> Evaluation (replacement)

In [3]:
def evaluation(gen, pop, decoded, scores, best, best_eval):
    for i in range(len(pop)):
        if scores[i] < best_eval:
            best, best_eval = pop[i], scores[i]
            print("- At the generation %d, obtained new best solution f(%s) = %f" % (gen,  decoded[i], scores[i]))
    return best, best_eval

In [4]:
def selection(pop, scores):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), 2):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

In [5]:
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

In [6]:
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

**Step 4.** combine all in binary-coded ganetic algoritm

In [7]:
def binary_coded_GA(f, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    

    # keep track of best solution
    best, best_eval = 0, f(decode(bounds, n_bits, pop[0]))
    
    # enumerate generations
    for gen in range(n_iter):
        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [f(d) for d in decoded]
        
        # check for new best solution
        best, best_eval = evaluation(gen, pop, decoded, scores, best, best_eval)
        

        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]


**Step 5.**  run the test Rosenbrock function search to check algorithm's reliability

In [8]:

best, score = binary_coded_GA(rosenbrock, ROSEN_BOUNDS, ROSEN_BITS, ROSEN_ITER, ROSEN_POP, ROSEN_CROSS, ROSEN_MUT)
print('Done!')
decoded = decode(ROSEN_BOUNDS, ROSEN_BITS, best)
print('f(%s) = %f' % (decoded, score))


- At the generation 0, obtained new best solution f([-0.02227783203125, 7.35443115234375]) = 5409.080831
- At the generation 0, obtained new best solution f([-0.274658203125, -5.15594482421875]) = 2738.360467
- At the generation 0, obtained new best solution f([0.7025146484375, 1.4794921875]) = 97.301266
- At the generation 0, obtained new best solution f([0.9271240234375, 0.49346923828125]) = 13.407479
- At the generation 4, obtained new best solution f([0.4339599609375, 0.47515869140625]) = 8.547973
- At the generation 6, obtained new best solution f([0.5413818359375, 0.56365966796875]) = 7.530893
- At the generation 7, obtained new best solution f([0.5413818359375, 0.4852294921875]) = 3.901924
- At the generation 7, obtained new best solution f([0.2288818359375, 0.16571044921875]) = 1.878846
- At the generation 8, obtained new best solution f([0.3045654296875, 0.17303466796875]) = 1.128030
- At the generation 9, obtained new best solution f([0.3045654296875, 0.1727294921875]) = 1.12

**Step 6.** Since rosenbrock works quite well, now we can run the objective function:

In [9]:
# perform the genetic algorithm search
best, score = binary_coded_GA(f, OBJ_BOUNDS, OBJ_BITS, OBJ_ITER, OBJ_POP, OBJ_CROSS, OBJ_MUT)
print('Done!')
decoded = decode(OBJ_BOUNDS, OBJ_BITS, best)
print('f(%s) = %f' % (decoded, score))

- At the generation 2, obtained new best solution f([4.96875, 2.015625]) = -36.440186
- At the generation 3, obtained new best solution f([4.96875, 1.015625]) = -39.377686
- At the generation 12, obtained new best solution f([4.984375, 1.078125]) = -39.414062
- At the generation 19, obtained new best solution f([4.984375, 1.015625]) = -39.657227
- At the generation 40, obtained new best solution f([4.984375, 1.0]) = -39.719238
Done!
f([4.984375, 1.0]) = -39.719238


## Task2 and Task3 Real-Coded Genetic Algorithm for continuous function ##

**Step1.** at the beginning we define new function and constraints for rcga algorithm